# Análise do AQE Python

Nesta análise, vamos avaliar a performance de tempo de resposta do AQE Python, assim como do ranking de documentos retornados pelo elasticsearch através da consulta expandida pelo AQE. Vamos utilizar as queries definidas na [base REGIS](https://github.com/Petroles/regis-collection/) para avaliação do AQE Python.

Para esta análise é necessário ter acesso ao AQE Python rodando, bem como ao elasticsearch, apontando as variáveis necessárias no `.env`.

## Dependências

In [ ]:
import json
import os
import re
import requests
import time
import xml.etree.ElementTree as ET
import pandas as pd
import plotly.express as px

from dotenv import load_dotenv

from utils.utils import create_metrics, create_ranking_dataset, \
    retrieve_from_elasticsearch

requests.packages.urllib3.util.connection.HAS_IPV6 = False
load_dotenv()

## Realizando consultas e calculando tempo de resposta

Agora vamos realizar as consultas das queries do REGIS para o AQE Python, calculando o tempo de resposta. Para cada query, vamos fazer o request 30 vezes, para identificar também a variação do tempo de resposta.

In [ ]:
with open("../data/regis_queries.json", "r") as json_file:
    regis_queries = json.loads(json_file.read())

certificate_path = "../PetrobrasCARootCorporativa.crt"

In [ ]:
number_of_requests = 30
max_expanded_terms = 5
aqe_base_url = os.getenv("AQE_URL")

df_data = list()
for i in range(number_of_requests):
    for query in regis_queries:
        query_title = query.get("title")
        query_id = query.get("query_id")
        url_query = f"{aqe_base_url}?query={query_title}&max_expanded_terms={max_expanded_terms}"
        start = time.time()
        response = requests.get(
            url_query,
            verify=certificate_path
        )
        end = time.time()
        response_time = (end - start) * 1000

        df_data.append(
            query | {
                "response": response.text,
                "response_time_ms": response_time,
                "num_original_terms": query_title.count(" ") + 1,
                "num_expanded_terms": len(re.findall("\^[0-1].[0-9]{3}", response.text)),
            }
        )

In [ ]:
response_df = pd.DataFrame(df_data)
response_df.head()

## Analisando tempo de resposta

Primeiramente vamos analisar o tempo de resposta das queries de uma maneira geral.

In [ ]:
fig = px.histogram(
    response_df, x="response_time_ms", nbins=15,
    title="Tempo de resposta do AQE Python",
    labels={
        "response_time_ms": "Tempo de resposta (ms)",
    }
).update_layout(
    yaxis_title_text="Contagem de requisições",
)
fig.show()

Podemos ver que a distribuição dos tempos de resposta tem uma maior frequência entre os 100 e 150 ms, com uma longa cauda a direita, que chega próximo dos 500 ms.

Vejamos agora o tempo de resposta para cada uma das queries.

In [ ]:
fig = px.box(
    response_df, x="query_id", y="response_time_ms",
    title="Tempo de resposta das queries",
    labels={
        "query_id": "Query ID",
        "response_time_ms": "Tempo de resposta (ms)",
    }
)
fig.show()

Podemos ver que o tempo de resposta tem uma variação considerável, em alguns casos variando próximo dos 200 ms.

Vejamos agora as medianas de cada query para ver a tendência central dos tempos de resposta de cada query, atenuando as variações.

In [ ]:
data_viz = response_df.groupby(
    "query_id"
).agg(
    {"response_time_ms": "median"}
).reset_index()

fig = px.bar(
    data_viz, x="query_id", y="response_time_ms",
    title="Tempo mediano de resposta das queries",
    labels={
        "query_id": "Query ID",
        "response_time_ms": "Mediana do tempo de resposta (ms)",
    }
).add_hline(
    y=response_df.response_time_ms.median(),
    annotation_text=f"{response_df.response_time_ms.median():.0f} ms"
)
fig.show()

Podemos ver que a mediana do tempo de resposta fica entre 150 e 200 ms e que mediana da Q14, Q26 e Q34 possui os maiores valores, acima dos 400 ms, enquanto a Q6 possui o menor tempo de resposta, próximo dos 45 ms.

## Realizando consultas no Elasticsearch

Agora vamos realizar as consultas ao elastic search e criar o dataset de validação, o qual possui informações do ground truth da base de dados REGIS.

In [ ]:
cfg = {
    "elasticsearch": {
        "url": os.getenv("ELASTIC_SEARCH_URL"),
        "index": os.getenv("ELASTIC_SEARCH_INDEX"),
        "username": os.getenv("ELASTIC_SEARCH_USERNAME"),
        "password": os.getenv("ELASTIC_SEARCH_PASSWORD"),
        "certificate": certificate_path
    }
}

queries = response_df.filter(
    items=["query_id", "response"]
).drop_duplicates(
).itertuples(
    index=False, name=None
)
queries = list(queries)

In [ ]:
ranking_result_df = retrieve_from_elasticsearch(queries, cfg, 24)
ranking_result_df.head()

In [ ]:
ground_truth = pd.read_csv("../data/regis_ground_truth.csv")
ground_truth.head()

In [ ]:
ranking_dataset = create_ranking_dataset(ranking_result_df, ground_truth)
ranking_dataset.head()

## Análise das consultas no Elasticsearch

Agora vamos criar as métricas para cada base de dados e fator e visualizar os resultados.

### Criando métricas

In [ ]:
metrics_df = create_metrics(ranking_dataset, groupby_columns=["query_id"])
metrics_df.head()

### Avaliando métricas

Vamos agora avaliar as métricas de ranking utilizando o NDCG (Normalized Discounted Cumulative Gain) como métrica de desempenho.

In [ ]:
fig = px.bar(
    metrics_df, x="query_id", y="ndcg",
    title="NDCG das queries",
    labels={
        "query_id": "Query ID",
        "ndcg": "NDCG (Normalized Discounted Cumulative Gain)",
    }
).add_hline(
    y=metrics_df.ndcg.mean(),
    annotation_text=f"NDCG médio {metrics_df.ndcg.mean():.4f}"
).update_layout(xaxis={"categoryorder":"total descending"})
fig.show()

Podemos ver que O NDCG médio foi de 81,31% e que apenas três das 34 queries (8,8%) obtiveram um NDCG abaixo dos 60%.

## Conclusão

Pudemos ver nesta análise que o tempo de resposta das queries para o AQE Python varia entre 0 e 500 ms, sendo o valor mais frequente fica entre 100 e 150 ms.
Vimos também que a métricas de ranking atingida com o AQE Python foi de 81,31%, utilizando o NDCG@24.